In [3]:
import hashlib   # Imports the hashlib module to perform SHA-256 hashing
import json      # Imports json to format and print data in JSON format
import time      # Imports time to use the current timestamp for blocks

# Define a class to represent each block in the blockchain
class Block:
    # Constructor method to initialize block attributes
    def __init__(self, index, domain, ip_address, timestamp, previous_hash):
        self.index = index                     # Position of the block in the chain
        self.domain = domain                   # Domain name stored in the block
        self.ip_address = ip_address           # IP address linked to the domain
        self.timestamp = timestamp             # Time when the block was created
        self.previous_hash = previous_hash     # Hash of the previous block
        self.hash = self.calculate_hash()      # Hash of the current block, calculated at creation

    # Method to calculate the SHA-256 hash of the block's contents
    def calculate_hash(self):
        # Concatenate all block attributes into a string
        block_string = f"{self.index}{self.domain}{self.ip_address}{self.timestamp}{self.previous_hash}"
        # Return the SHA-256 hash of the string
        return hashlib.sha256(block_string.encode()).hexdigest()

    # Convert block data to a dictionary format (useful for JSON output)
    def to_dict(self):
        return {
            'index': self.index,
            'domain': self.domain,
            'ip_address': self.ip_address,
            'timestamp': self.timestamp,
            'previous_hash': self.previous_hash,
            'hash': self.hash
        }

# Define a class to manage the blockchain (a list of blocks)
class Blockchain:
    # Constructor initializes the chain with the genesis block
    def __init__(self):
        self.chain = []                # List to store the blocks
        self.create_genesis_block()    # Call to create the first block in the chain

    # Method to create the first block manually (hardcoded data)
    def create_genesis_block(self):
        genesis_block = Block(0, "genesis", "0.0.0.0", time.time(), "0")  # Create genesis block with fixed data
        self.chain.append(genesis_block)                                  # Add it to the chain

    # Method to add a new block to the chain
    def add_block(self, domain, ip_address):
        last_block = self.chain[-1]   # Get the most recent block
        # Create a new block with the next index, domain, IP, timestamp, and previous hash
        new_block = Block(len(self.chain), domain, ip_address, time.time(), last_block.hash)
        self.chain.append(new_block)  # Append the new block to the chain
        insert_block(new_block)       # Call external function to insert block (assumed defined elsewhere)

    # Method to validate the integrity of the entire chain
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):               # Loop through all blocks except genesis
            curr = self.chain[i]                          # Current block
            prev = self.chain[i - 1]                      # Previous block
            if curr.hash != curr.calculate_hash():        # Recalculate hash and compare
                return False
            if curr.previous_hash != prev.hash:           # Check hash linkage between blocks
                return False
        return True                                       # If all checks pass, the chain is valid

    # Method to print the entire blockchain in JSON format
    def print_chain(self):
        for block in self.chain:
            print(json.dumps(block.to_dict(), indent=4))  # Print block dictionary nicely formatted


In [4]:
bc = Blockchain()  
# Creates a new instance of the Blockchain class
# This automatically creates and appends the genesis block to the chain

bc.add_block("example.com", "192.168.1.5")  
# Adds a new block to the blockchain with domain "example.com" and IP "192.168.1.5"
# It calculates the new block’s hash, links it to the last block, and calls insert_block()

bc.add_block("openai.com", "10.0.0.1")  
# Adds another new block with domain "openai.com" and IP "10.0.0.1"
# This block links to the one added just before it

bc.print_chain()  
# Prints out all the blocks in the blockchain in a formatted JSON view
# Each block shows: index, domain, IP address, timestamp, previous hash, and current hash

NameError: name 'insert_block' is not defined

In [ ]:
def init_db():
    conn = sqlite3.connect("dns_blockchain.db")  
    # Connects to (or creates) an SQLite database file named "dns_blockchain.db"

    c = conn.cursor()  
    # Creates a cursor object to execute SQL commands

    c.execute('''CREATE TABLE IF NOT EXISTS blocks (
        index_no INTEGER,
        domain TEXT,
        ip_address TEXT,
        timestamp REAL,
        previous_hash TEXT,
        hash TEXT
    )''')  
    # Creates a table called `blocks` if it doesn't already exist
    # with columns for all block attributes

    conn.commit()  
    # Saves the changes to the database

    conn.close()  
    # Closes the database connection


In [ ]:
init_db()    # Step 1: Initialize the database and table

bc = Blockchain()    # Step 2: Create a new blockchain (includes genesis block)

bc.add_block("example.com", "192.168.1.5")   # Step 3: Add first custom block
bc.add_block("openai.com", "10.0.0.1")       # Step 4: Add second custom block

bc.print_chain()     # Step 5: Print all blocks in the chain


In [ ]:
conn = sqlite3.connect("dns_blockchain.db")  
# Connects to the SQLite database file named "dns_blockchain.db"
# If the file doesn't exist, it will be created

c = conn.cursor()  
# Creates a cursor object to execute SQL queries

for row in c.execute("SELECT * FROM blocks"):  
    # Executes a SQL SELECT query to fetch all rows from the "blocks" table
    # Loops through each row returned by the query
    print(row)  
    # Prints the row to the console (each row is a tuple of block data)

conn.close()  
# Closes the database connection

In [ ]:
def resolve_domain(domain_name):
    conn = sqlite3.connect("dns_blockchain.db")  
    # Connect to the SQLite database named "dns_blockchain.db"

    c = conn.cursor()  
    # Create a cursor object to execute SQL commands

    c.execute("SELECT ip_address, timestamp FROM blocks WHERE domain = ?", (domain_name,))  
    # Use a parameterized query to fetch IP and timestamp for the given domain
    # Prevents SQL injection attacks

    result = c.fetchone()  
    # Fetch the first matching result (returns None if no match found)

    conn.close()  
    # Close the database connection

    if result:
        ip, ts = result  
        # Unpack the tuple into IP address and timestamp
        print(f"Domain: {domain_name}\nIP Address: {ip}\nRegistered At: {time.ctime(ts)}")  
        # Print the domain resolution result in a user-friendly format
    else:
        print("Domain not found in the blockchain.")  
        # Print this message if the domain does not exist in the DB


In [10]:
resolve_domain("notexist.com")


NameError: name 'resolve_domain' is not defined

In [29]:
resolve_domain("example.com")


Domain: example.com
IP Address: 192.168.1.5
Registered At: Tue May 20 15:18:23 2025


In [31]:
def show_all_domains():
    conn = sqlite3.connect("dns_blockchain.db")
    c = conn.cursor()
    for row in c.execute("SELECT domain, ip_address FROM blocks"):
        print(f"{row[0]} → {row[1]}")
    conn.close()


In [33]:
show_all_domains()


example.com → 192.168.1.5
openai.com → 10.0.0.1
